In [ ]:
import os
import json
import subprocess
import pandas as pd
import tensorflow as tf
from tensor2tensor.layers import common_layers

## path and parameters definitions

In [ ]:
WORKSPACE_PATH = '/tmp/tes-workspace'
os.makedirs(WORKSPACE_PATH, exist_ok=True)
DATA_PATH = os.path.join(WORKSPACE_PATH, 'data')
os.makedirs(DATA_PATH, exist_ok=True)
TRAIN_FILEPATH = os.path.join(DATA_PATH, 'train.txt')
EVAL_FILEPATH = os.path.join(DATA_PATH, 'eval.txt')
MAX_LEN = 200
MODEL_DIR = os.path.join(WORKSPACE_PATH, 'test_model')
MODEL_EXPORTER_DIR = '{}_exporter'.format(MODEL_DIR)
TRAIN_STEPS = 2000

## prepare data

Here we use the pre-processed data from `tf.keras.datasets.imdb`. We dump them to disk in order to demonstrate how to use TensorFlow high-level API using custom files.

In [ ]:
# prepare data
def _pad_and_write(X, y, max_len, filepath):
    def _pad(a_list):
        length = len(a_list)
        padded = a_list
        if length < max_len:
            padding = max_len - length
            padded =  a_list + [0]*padding
        return padded
            
    with open(filepath, 'w') as fp:
        fp.writelines([
            '{}|{}\n'.format(
                ','.join(map(str,_pad(tokens))),
                label
            )
            for tokens, label in zip(X,y)
        ])
   
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(
    path='imdb.npz',
    num_words=None,
    skip_top=0,
    maxlen=MAX_LEN,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)
_pad_and_write(X_train, y_train, MAX_LEN, TRAIN_FILEPATH)
_pad_and_write(X_test, y_test, MAX_LEN, EVAL_FILEPATH)

## define dataset

Data are feeded to the model starting from a `tf.data.TextLineDataset`.

In [ ]:
# define dataset
def _process_line(line):
    """Process a line to create input tensors and labels."""
    line_split = tf.string_split([line], '|')

    return (
        {
            'tokens': tf.reshape(
                tf.string_to_number(
                    tf.string_split([line_split.values[0]], ',').values,
                    out_type=tf.int32
                ),
                [MAX_LEN]
            )
        },
        tf.string_to_number(line_split.values[1], out_type=tf.int32)
    )


def generate_dataset(filepath):
    """Generate a tf.Dataset given a filepath containing one observation per line."""
    return tf.data.TextLineDataset(
        filepath
    ).map(
        lambda line: _process_line(line)
    )

## define model function

Here we define a dummy model for text classification.

In [ ]:
# define model
def model_fn(
    features, labels, mode, params
):
    """
    Model function definition.
    
    The model is quite dummy but is used for demonstration
    purposes.
    """
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    # NOTE: dropout considered only during training
    dropout = 0.0
    if is_training:
        dropout = params.get('dropout', 0.0)
    
    tokens = features['tokens']

    embedded_tokens = common_layers.embedding(
        tokens,
        params['vocabulary_size'],
        params['embedding_size'],
        name='embedding'
    )

    convolutions = common_layers.conv1d(
        embedded_tokens, params['filters'], params['kernel_size']
    )
    
    flattened = tf.layers.flatten(convolutions)
    hidden = tf.layers.dropout(
        tf.layers.dense(
            flattened, params['dense_hidden_size'],
            activation=params.get('activation', tf.nn.relu),
            name='dense_hidden'
        ),
        rate=dropout,
        training=is_training,
        name='dropout_dense_hidden'
    )
    
    
    logits = tf.layers.dense(
        hidden, params['number_of_classes'], name='logits'
    )
    predictions = tf.argmax(logits, axis=1)
    
    prediction_dict = {
        'predictions': predictions
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, predictions=prediction_dict
        )
    
    # loss
    one_hot_labels = tf.one_hot(labels, depth=params['number_of_classes'])
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=one_hot_labels, logits=logits
    )
    loss = tf.reduce_mean(cross_entropy)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(
            params.get('learning_rate', 0.001),
            tf.train.get_global_step(),
            decay_steps=params.get('decay_steps', 3000),
            decay_rate=params.get('decay_rate', 0.96)
        )
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        return tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions, loss=loss,
            train_op=optimizer.minimize(loss, tf.train.get_global_step())
        )
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss
        )


## define functions used to feed data in train/eval/serving

In [ ]:
# learning functions
def train_input_fn(params):
    """Train function given params."""
    batch_size = params['batch_size']
    filepath = params['train_filepath']
    dataset = generate_dataset(
        filepath
    ).cache().shuffle(
        buffer_size=params.get('buffer_size', 50000)
    ).batch(
        batch_size, drop_remainder=True
    ).repeat()
    features, labels = dataset.make_one_shot_iterator().get_next()
    return features, labels


def eval_input_fn(params):
    """Eval function given params."""
    batch_size = params['batch_size']
    filepath = params['eval_filepath']
    dataset = generate_dataset(
        filepath
    ).batch(
        batch_size, drop_remainder=True
    )
    features, labels = dataset.make_one_shot_iterator().get_next()
    return features, labels


def serving_input_with_params_fn(params):
    """Serving function given params."""
    features = {
        'tokens': tf.placeholder(
            tf.int32,
            [None, MAX_LEN]
        )
    }
    return tf.estimator.export.ServingInputReceiver(features, features)

## define parameters and model exporter

In [ ]:
# parameters
params = {
    'batch_size': 32,
    'train_filepath': TRAIN_FILEPATH,
    'eval_filepath': EVAL_FILEPATH,
    'vocabulary_size': 88585,
    'embedding_size': 64,
    'kernel_size': 5,
    'filters': 8,
    'dense_hidden_size': 16,
    'number_of_classes': 2,
    'dropout': 0.75
}
# define exporter
exporter = tf.estimator.LatestExporter(
    MODEL_EXPORTER_DIR,
    lambda: serving_input_with_params_fn(params),
    exports_to_keep=None
)

## define a `tf.estimator.Estimator`

In [ ]:
# define the estimator
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params
)

## train the model

In [ ]:
# define training and eval specifications
train_spec = tf.estimator.TrainSpec(
   input_fn=train_input_fn,
   max_steps=TRAIN_STEPS
)
eval_spec = tf.estimator.EvalSpec(
    input_fn=eval_input_fn,
    throttle_secs=60,
    exporters=exporter
)
tf.estimator.RunConfig.save_checkpoints_steps = min(300, TRAIN_STEPS // 3)
tf.estimator.RunConfig.save_checkpoints_secs = None

In [ ]:
# check the training on `tensorboard`
print(
    'Follow training evolution with:\ntensorboard --logdir {}\n'.format(estimator.model_dir)
)

In [ ]:
# train and evaluate
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## test the exported model with `tensorflow/serving` Docker image

Here we assume that `docker` and `curl` are installed.
Before running this section make sure the `tensorflow/serving` image is pulled

In [ ]:
print(
    'Serve the model with:\ndocker run -p 8501:8501 '
    '--mount type=bind,source={},target=/models/tes '
    '-e MODEL_NAME=tes -t tensorflow/serving\n'.format(exporter.name)
)

In [ ]:
# prepare query
eval_data = pd.read_csv(EVAL_FILEPATH, sep='|', names=['tokens', 'label'])
index = 1
tokens = list(map(int, eval_data['tokens'][index].split(',')))

query = json.dumps({ 'instances': [{
    'tokens': tokens
}]}, indent=1)

In [ ]:
# query the model deployed
json.loads(subprocess.check_output(
    ['curl', '-d', '{}'.format(query), 'http://localhost:8501/v1/models/tes:predict']
))